In [ ]:
import pandas as pd
import requests
import json
import os
from io import BytesIO
from ISR.models import RDN;
import numpy as np
from PIL import Image
import time
from YDKReader import Reader
from IPython.display import clear_output

database_filename = "database.csv"
database_folder = "./Images Database/"

database = pd.DataFrame()

## Part 1: Update the Database

In [ ]:
def update_database():
    '''
    Update cards database for newer cards.
    '''
    print("Updating Database....")
    response = requests.get("https://db.ygoprodeck.com/api/v7/cardinfo.php")
    if response.status_code == 200:
        print("Success")
        card = response.json()
        df = pd.json_normalize(card['data'], record_path=['card_images'], meta=['name'])
        df = df[1:]
        df['upscaled_image'] = " "
        df = df.set_index('id')
        df.index = df.index.astype(str)
        #old_df = pd.read_csv(database_filename, index_col="id")
        saved_images = os.listdir('./Images Database/')
        # saved_images = old_df['upscaled_image']
        for i in saved_images:
            df.at[i.split('.png')[0], 'upscaled_image'] = './Images Database/' + i
        df.to_csv('./' + database_filename)
        return df


## Part 2: Import the Database

In [ ]:
def import_database():
    '''
    Import database. A dataframe object is then returned
    '''
    database = pd.read_csv(database_filename, index_col="id")
    database.index = database.index.astype(str)
    #database.head()
    return database

#### Enable Qt File explorer to fetch deck file

In [ ]:
%gui qt

from PyQt5.QtWidgets import QFileDialog

def gui_fname(dir=None):
    '''
    Select a file via a dialog and return the file name.
    '''
    if dir is None: dir ='./'
    fname = QFileDialog.getOpenFileName(None, "Select data file...", 
                dir, filter="All files (*);; SM Files (*.sm)")
    return fname[0]

## Part 3: Image setup fuctions

In [ ]:
def get_image(card_image:str):
    '''
    Create a GET request to fetch the card image. An image is then returned in bytes format.
    '''
    image = requests.get(card_image, timeout=120)
    image = requests.get(card_image)
    image = Image.open(BytesIO(image.content))
    return image

In [ ]:
def upscale_image_local(card_image:str):
    '''
    Upscale a given image url using RDN Super Resolution neural network. The image file path is then returned.
    '''    
    image = get_image(card_image)
    lr_img = np.array(image)
    rdn = RDN(weights='noise-cancel');
    sr_image = rdn.predict(lr_img, by_patch_of_size=50)
    result = Image.fromarray(sr_image)
    file_path = database_folder + str(id) + ".png"
    result.save(file_path, "PNG")
    
    return file_path

In [ ]:
def process_card(id:str):
    '''
    Start processing a card given its id. The image file is then returned.
    '''
    proccessed = database.at[id, 'upscaled_image'] != " "

    image_path = ""
    if proccessed == False:
        clear_output(wait=True)
        print(database.at[id, 'name'])
        print("Image not found.\nProccessing now...")
        image_path = upscale_image_local(database.at[id, 'image_url'])
        database.at[id, 'upscaled_image'] = image_path
        database.to_csv("./" + database_filename)
    else:
        #get the image directly
        image_path = database.at[id, 'upscaled_image']
        print("Image found at " + image_path)

    return Image.open(image_path)

## Part 4: Create .odg file and add the deck cards to it

In [ ]:
from ODGEditor import ODGEditor
import math
def create_doc_file(folder_name, deck:dict, back_sleev=""):
    '''
    Create a .odg file placing the file in 'folder_name', 
    adding cards in 'deck' and placing the desired sleeve (optional)
    '''
    odgEditor = ODGEditor(create_path=folder_name, deck=deck.get_result(), back_sleev=back_sleev)

    page_number = math.ceil(sum(deck.get_result().values()) / 9.0)
    odgEditor.add_pages(page_number)
    odgEditor.insert_cards()
    odgEditor.create_new_doc()

    odgEditor.copy_card_files()

### Part 5: Execution of code

In [ ]:
# choosing deck to work on
file_name = gui_fname()
file_name = './YDK Files/' + file_name.split('/')[-1]
folder_name = file_name.split('.ydk')[0]

deck = Reader(file_name)   # The deck data is read and a new folder for the deck is created
back_sleev = ""

# importing a deck sleeve if desired
prompt = input("Do you want a back sleev? (y/n)\n")
if prompt == 'y':
    back_sleev = gui_fname()

In [ ]:
# All of the deck cooking is done here
database = import_database()

for i, j in deck.get_result().items():
    id = str(i)

    if id not in database.index: database = update_database()
    
    card_name = database.at[id, 'name']
    card_image = database.at[id, 'image_url']
    print({card_name : card_image})
    
    image = process_card(id)   
    #image.save("{}/{} x{}.png".format(folder_name, "({})".format(id), j), "PNG")
create_doc_file(folder_name, deck, back_sleev=back_sleev)

print("Deck Done!")

## Cloud API calling (not currently used due to limited quota)

#### Upscale using the replicate Super Resolution neural network

In [ ]:
def upscale_image_replicate():
    post_data = {
        "version": "9d91795e944f3a585fa83f749617fc75821bea8b323348f39cf84f8fd0cbc2f7",
        "input" : {"image" : card_image}
    }
    post_hearders = {
        "Authorization" : "Token [my token :)]",
        "Content-Type" : "application/json"
    }

    response = requests.post('https://api.replicate.com/v1/predictions', data=json.dumps(post_data), headers=post_hearders)
    if response.status_code != 201:
        print(response.status_code)
        print(response.content)
    else:
        print(response.json())
        
    #get the processed card after upscaling
    response_dict = dict(response.json())

    # wait until the process is finished
    time.sleep(20)
    output_image = requests.get(response_dict['urls']['get'], headers=post_hearders)

    print(output_image.status_code)
    print(output_image.content)

    #get the image after prcoessing it
    img_url = dict(output_image.json())['output'][0]['file']
    
    #caching the image link
    database.at[id, "upscaled_image"] = img_url
    database.to_csv('./' + database_filename)


#### Upscale using the waifu2x Super Resolution (model uploaded on DeepAI)

In [ ]:
def upscale_image_waifu2x():
   response = requests.post(
    "https://api.deepai.org/api/waifu2x",
    data={
        'image': card_image,
    },
    headers={'api-key': 'quickstart-QUdJIGlzIGNvbWluZy4uLi4K'}
   )
   time.sleep(10)
   print(response.json())
   img = response.json()['output_url']
   import urllib.request
   file_path = database_folder + str(id)
   urllib.request.urlretrieve(img, file_path + ".png")
   return file_path